In [83]:
##import necessary packages

import pandas as pd
from bs4 import BeautifulSoup
import regex
import re
import numpy
import requests
import pprint
from pathlib import Path

In [2]:
##Create function that removes unneeded string elements

def strip_word(word: str | None) -> str | None:
    if word == None:
        return None
    letters = []
    word = word.replace('[d]','')
    word = word.replace('[c]','')
    for i in word:
       if i.isalpha() or i.isspace():
            letters.append(i)
    word = ''.join(letters)
    word = word.strip()
    return word


In [3]:
def remove_banned(word: str, banned: list) -> str:
    translation_table = str.maketrans('', '', ''.join(banned))
    removed_char = word.translate(translation_table)
    print(removed_char)
    
         

In [4]:
def is_empty(word: str) -> bool:
    return word == ''
def transform_row(word: str) -> bool:
    if word == '—':
        return None
    else:
        return word

In [5]:
def ref_filter(word: str | None) -> str | None:
    result = ''
    if word == None:
        return None
    for letter in word:
        if letter.isalnum() or letter == '/':
            result += letter
        else:
            return result
    return result
            

In [6]:
url = 'https://en.wikipedia.org/wiki/Major_League_Baseball_Most_Valuable_Player_Award'
html = requests.get(url)


In [7]:
soup = BeautifulSoup(html.content, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [8]:
banned = ['American League winner','Year', 'Team', 'Position', 'National League winner']
tables = soup.select('.wikitable')[1:4]
col_names = []
data = {}
doube_row = False
table_num = 0
for table in tables:
    table_num += 1
    headers = table.select('tr th')
    #Below logic gets header fields
    for header in headers:
        stripped_header = header.text.strip()
        if header.get('scope') == 'col' and stripped_header not in data.keys() and stripped_header !='Ref':
            data[stripped_header] = []
    #Get all row values
    index = 0
    if table_num <= 2:
        for row in table.select('tbody tr td, th'):
            if row.get('title') != ''  and row.get('scope') != 'col' and row.get_text() != ''and row.get_text() != 'Ref' and row.get('role') != 'columnheader button':
                text = transform_row(row.text.strip())
                if index == 0:
                    data['Year'].append(text)
                    index += 1
                elif index == 1:
                    data['American League winner'].append(strip_word(text))
                    index += 1
                elif index == 2:
                    data['Team'].append(strip_word(text))
                    index += 1
                elif index == 3:
                    data['Position'].append(ref_filter(text))
                    index += 1
                elif index == 4:
                    data['National League winner'].append(strip_word(text))
                    index += 1
                elif index == 5:
                    data['Team'].append(strip_word(text))
                    index += 1
                elif index == 6:
                    index += 1
                    data['Position'].append(ref_filter(text))
                else:
                    index = 0
    else:
        for row in table.select('tbody tr td, th td'):
            if row.get('title') != ''  and row.get('scope') != 'col' and row.get_text() != ''and row.get_text() != 'Ref' and row.get('role') != 'columnheader button':
                text = transform_row(row.text.strip())
                #exception to account for a change in formatting of the table which upsets the index
                if row.text == 'Willie Stargell†[d]':
                    index = 4
                    data['Year'].append('1979')
                    data['American League winner'].append(None)
                    data['Position'].append(None)
                    data['Team'].append(None)
                if row.text == '1980':
                    index = 0
                if index == 0:
                    data['Year'].append(text)
                    index += 1
                elif index == 1:
                    data['American League winner'].append(strip_word(text))
                    index += 1
                elif index == 2:
                    data['Team'].append(strip_word(text))
                    index += 1
                elif index == 3:
                    data['Position'].append(ref_filter(text))
                    index += 1
                elif index == 4:
                    data['National League winner'].append(strip_word(text))
                    index += 1
                elif index == 5:
                    data['Team'].append(strip_word(text))
                    index += 1
                elif index == 6:
                    index += 1
                    data['Position'].append(ref_filter(text))
                else:
                    index = 0
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(data)

      

            
            
            
            


{ 'American League winner': [ 'Ty Cobb',
                              'Tris Speaker',
                              'Walter Johnson',
                              'Eddie Collins',
                              'George Sisler',
                              'Babe Ruth',
                              'Walter Johnson',
                              'Roger Peckinpaugh',
                              'George Burns',
                              'Lou Gehrig',
                              'Mickey Cochrane',
                              None,
                              'Lefty Grove',
                              'Jimmie Foxx',
                              'Jimmie Foxx',
                              'Mickey Cochrane',
                              'Hank Greenberg',
                              'Lou Gehrig',
                              'Charlie Gehringer',
                              'Jimmie Foxx',
                              'Joe DiMaggio',
                              'Hank 

In [9]:
##Ensure that columns contain proper amount of values
def check_row_length(data_set: dict[str,list[str|None]]) -> None:
    for key in data_set.keys():
        key_length = len(data_set[key])
        print(f'{key} contains {key_length} items') 

In [10]:

check_row_length(data)

Year contains 104 items
American League winner contains 104 items
Team contains 208 items
Position contains 208 items
National League winner contains 104 items


In [11]:
for table in tables:
    for row in table.select('tbody tr td, th'):
        print(row.text)




Year

American League winner

Team

Position

National League winner

Team

Position

Ref

1911

Ty Cobb§

Detroit Tigers
OF

Frank Schulte

Chicago Cubs
OF
[19]

1912

Tris Speaker

Boston Red Sox
OF

Larry Doyle

New York Giants
2B
[20]

1913

Walter Johnson

Washington Senators
RHP

Jake Daubert

Brooklyn Superbas
1B
[21]

1914

Eddie Collins

Philadelphia Athletics
2B

Johnny Evers

Boston Braves
2B
[22]

Year

American League winner

Team

Position

National League winner

Team

Position

Ref

1922

George Sisler

St. Louis Browns
1B

—

—
—
[26]

1923

Babe Ruth§

New York Yankees
OF

—

—
—
[27]

1924

Walter Johnson (2)

Washington Senators
RHP

Dazzy Vance

Brooklyn Robins
RHP
[28]

1925

Roger Peckinpaugh

Washington Senators
SS

Rogers Hornsby

St. Louis Cardinals
2B
[29]

1926

George Burns

Cleveland Indians
1B

Bob O'Farrell

St. Louis Cardinals
C
[30]

1927

Lou Gehrig

New York Yankees
1B

Paul Waner

Pittsburgh Pirates
OF
[31]

1928

Mickey Cochrane

Philadelphia Athle

In [12]:
##organize players and positions into their respective league
American_league_pos = []
National_league_pos = []
American_league_team = []
National_league_team = []
for i, pos in enumerate(data['Position']):
    if i%2 == 0:
        American_league_pos.append(pos)
    else:
        National_league_pos.append(pos)
for i, pos in enumerate(data['Team']):
    if i%2 == 0:
        American_league_team.append(pos)
    else:
        National_league_team.append(pos)
print(National_league_team)

['Chicago Cubs', 'New York Giants', 'Brooklyn Superbas', 'Boston Braves', None, None, 'Brooklyn Robins', 'St Louis Cardinals', 'St Louis Cardinals', 'Pittsburgh Pirates', 'St Louis Cardinals', 'Chicago Cubs', 'St Louis Cardinals', 'Philadelphia Phillies', 'New York Giants', 'St Louis Cardinals', 'Chicago Cubs', 'New York Giants', 'St Louis Cardinals', 'Cincinnati Reds', 'Cincinnati Reds', 'Cincinnati Reds', 'Brooklyn Dodgers', 'St Louis Cardinals', 'St Louis Cardinals', 'St Louis Cardinals', 'Chicago Cubs', 'St Louis Cardinals', 'Boston Braves', 'St Louis Cardinals', 'Brooklyn Dodgers', 'Philadelphia Phillies', 'Brooklyn Dodgers', 'Chicago Cubs', 'Brooklyn Dodgers', 'New York Giants', 'Brooklyn Dodgers', 'Brooklyn Dodgers', 'Milwaukee Braves', 'Chicago Cubs', 'Chicago Cubs', 'Pittsburgh Pirates', 'Cincinnati Reds', 'Los Angeles Dodgers', 'Los Angeles Dodgers', 'St Louis Cardinals', 'San Francisco Giants', 'Pittsburgh Pirates', 'St Louis Cardinals', 'St Louis Cardinals', 'San Francisco 

In [13]:
##Create a new dictionary from new list
from hashlib import new

new_dict ={'Year':[],'American League winner':[],'American League Team':[], 'American League Position':[], 'National League winner':[], 'National League Team':[], 'National League Position':[]}
for key in data.keys():
    if key in new_dict.keys():
        new_dict[key] = data[key]
new_dict['American League Team'] = American_league_team
new_dict['National League Team'] = National_league_team
new_dict['National League Position'] = National_league_pos
new_dict['American League Position'] = American_league_pos

In [14]:
pp.pprint(new_dict)

{ 'American League Position': [ 'OF',
                                'OF',
                                'RHP',
                                '2B',
                                '1B',
                                'OF',
                                'RHP',
                                'SS',
                                '1B',
                                '1B',
                                'C',
                                None,
                                'LHP',
                                '1B',
                                '1B',
                                'C',
                                '1B',
                                '1B',
                                '2B',
                                '1B',
                                'OF',
                                'OF',
                                'OF',
                                '2B',
                                'RHP',
                                'LHP',
         

In [15]:
##Display new data frame
new_df = pd.DataFrame.from_dict(new_dict)
pd.set_option('display.max_row', None)

In [16]:
display(new_df)

,Year,American League winner,American League Team,American League Position,National League winner,National League Team,National League Position
0,1911,Ty Cobb,Detroit Tigers,OF,Frank Schulte,Chicago Cubs,OF
1,1912,Tris Speaker,Boston Red Sox,OF,Larry Doyle,New York Giants,2B
2,1913,Walter Johnson,Washington Senators,RHP,Jake Daubert,Brooklyn Superbas,1B
3,1914,Eddie Collins,Philadelphia Athletics,2B,Johnny Evers,Boston Braves,2B
4,1922,George Sisler,St Louis Browns,1B,None,None,None
5,1923,Babe Ruth,New York Yankees,OF,None,None,None
6,1924,Walter Johnson,Washington Senators,RHP,Dazzy Vance,Brooklyn Robins,RHP
7,1925,Roger Peckinpaugh,Washington Senators,SS,Rogers Hornsby,St Louis Cardinals,2B
8,1926,George Burns,Cleveland Indians,1B,Bob OFarrell,St Louis Cardinals,C
9,1927,Lou Gehrig,New York Yankees,1B,Paul Waner,Pittsburgh Pirates,OF


In [17]:
players_american = new_df['American League winner'].tolist()
players_national = new_df['National League winner'].tolist()
players = players_american + players_national
##Grab link endings so that they can be iterated through
player_links = []
for table in tables:
    for i in table.find_all('a'):
        text = transform_row(i.text.strip())
        if strip_word(text) in players:
            player_links.append(i['href'])
        
         

In [18]:
players_american = new_df['American League winner'].tolist()
players_national = new_df['National League winner'].tolist()
players = players_american + players_national
print(players)

['Ty Cobb', 'Tris Speaker', 'Walter Johnson', 'Eddie Collins', 'George Sisler', 'Babe Ruth', 'Walter Johnson', 'Roger Peckinpaugh', 'George Burns', 'Lou Gehrig', 'Mickey Cochrane', None, 'Lefty Grove', 'Jimmie Foxx', 'Jimmie Foxx', 'Mickey Cochrane', 'Hank Greenberg', 'Lou Gehrig', 'Charlie Gehringer', 'Jimmie Foxx', 'Joe DiMaggio', 'Hank Greenberg', 'Joe DiMaggio', 'Joe Gordon', 'Spud Chandler', 'Hal Newhouser', 'Hal Newhouser', 'Ted Williams', 'Joe DiMaggio', 'Lou Boudreau', 'Ted Williams', 'Phil Rizzuto', 'Yogi Berra', 'Bobby Shantz', 'Al Rosen', 'Yogi Berra', 'Yogi Berra', 'Mickey Mantle', 'Mickey Mantle', 'Jackie Jensen', 'Nellie Fox', 'Roger Maris', 'Roger Maris', 'Mickey Mantle', 'Elston Howard', 'Brooks Robinson', 'Zoilo Versalles', 'Frank Robinson', 'Carl Yastrzemski', 'Denny McLain', 'Harmon Killebrew', 'Boog Powell', 'Vida Blue', 'Dick Allen', 'Reggie Jackson', 'Jeff Burroughs', 'Fred Lynn', 'Thurman Munson', 'Rod Carew', 'Jim Rice', 'Don Baylor', None, 'George Brett', 'Roll

In [19]:

print((player_links))

['/wiki/Ty_Cobb', '/wiki/Frank_Schulte', '/wiki/Tris_Speaker', '/wiki/Larry_Doyle_(baseball)', '/wiki/Walter_Johnson', '/wiki/Jake_Daubert', '/wiki/Eddie_Collins', '/wiki/Johnny_Evers', '/wiki/George_Sisler', '/wiki/Babe_Ruth', '/wiki/Walter_Johnson', '/wiki/Dazzy_Vance', '/wiki/Roger_Peckinpaugh', '/wiki/Rogers_Hornsby', '/wiki/George_Burns_(first_baseman)', '/wiki/Bob_O%27Farrell', '/wiki/Lou_Gehrig', '/wiki/Paul_Waner', '/wiki/Mickey_Cochrane', '/wiki/Jim_Bottomley', '/wiki/Rogers_Hornsby', '/wiki/Lefty_Grove', '/wiki/Frankie_Frisch', '/wiki/Jimmie_Foxx', '/wiki/Chuck_Klein', '/wiki/Jimmie_Foxx', '/wiki/Carl_Hubbell', '/wiki/Mickey_Cochrane', '/wiki/Dizzy_Dean', '/wiki/Hank_Greenberg', '/wiki/Gabby_Hartnett', '/wiki/Lou_Gehrig', '/wiki/Carl_Hubbell', '/wiki/Charlie_Gehringer', '/wiki/Joe_Medwick', '/wiki/Jimmie_Foxx', '/wiki/Ernie_Lombardi', '/wiki/Joe_DiMaggio', '/wiki/Bucky_Walters', '/wiki/Hank_Greenberg', '/wiki/Frank_McCormick', '/wiki/Joe_DiMaggio', '/wiki/Dolph_Camilli', '/wi

In [20]:
print(players)

['Ty Cobb', 'Tris Speaker', 'Walter Johnson', 'Eddie Collins', 'George Sisler', 'Babe Ruth', 'Walter Johnson', 'Roger Peckinpaugh', 'George Burns', 'Lou Gehrig', 'Mickey Cochrane', None, 'Lefty Grove', 'Jimmie Foxx', 'Jimmie Foxx', 'Mickey Cochrane', 'Hank Greenberg', 'Lou Gehrig', 'Charlie Gehringer', 'Jimmie Foxx', 'Joe DiMaggio', 'Hank Greenberg', 'Joe DiMaggio', 'Joe Gordon', 'Spud Chandler', 'Hal Newhouser', 'Hal Newhouser', 'Ted Williams', 'Joe DiMaggio', 'Lou Boudreau', 'Ted Williams', 'Phil Rizzuto', 'Yogi Berra', 'Bobby Shantz', 'Al Rosen', 'Yogi Berra', 'Yogi Berra', 'Mickey Mantle', 'Mickey Mantle', 'Jackie Jensen', 'Nellie Fox', 'Roger Maris', 'Roger Maris', 'Mickey Mantle', 'Elston Howard', 'Brooks Robinson', 'Zoilo Versalles', 'Frank Robinson', 'Carl Yastrzemski', 'Denny McLain', 'Harmon Killebrew', 'Boog Powell', 'Vida Blue', 'Dick Allen', 'Reggie Jackson', 'Jeff Burroughs', 'Fred Lynn', 'Thurman Munson', 'Rod Carew', 'Jim Rice', 'Don Baylor', None, 'George Brett', 'Roll

In [21]:
player_name = player_soup.find('span').text
print(player_name)

NameError: name 'player_soup' is not defined

In [ ]:

start = player_soup.find('th', text = 'MLB statistics').parent
print(start)

AttributeError: 'NoneType' object has no attribute 'parent'

In [ ]:
player_dict = {}
for tr in start.find_next_siblings('tr'):
    if tr.find('th', text = 'Teams') or tr.find('a') is None:
        break
    print(type(tr.find('a')))
    key = tr.find('a').text
    value = tr.find('td').text
    player_dict[key] = value
    print(tr.find('a').text)
    print(tr.find('td').text)
    
    

<class 'bs4.element.Tag'>
Batting average
.296
<class 'bs4.element.Tag'>
Hits
2,844
<class 'bs4.element.Tag'>
Home runs
311
<class 'bs4.element.Tag'>
Runs batted in
1,332


In [ ]:
print(player_dict)

{'Batting average': '.296', 'Hits': '2,844', 'Home runs': '311', 'Runs batted in': '1,332'}


In [ ]:
def table_exception():
    for i in start.find_next_siblings():    
        stat_text = i.find('th',)
        stat_num =i.find('td')
        if stat_text != None and stat_num != None:
            print(stat_text.text)
            print(stat_num.text)

In [73]:
player_list = []
for i in range(len(player_links)):
    player_url = 'https://en.wikipedia.org' + str(player_links[i])
    player_html = requests.get(player_url)
    player_soup = BeautifulSoup(player_html.content, 'html.parser')
    if player_soup.find('span', class_ = 'note') == None:
        player_soup.find('span', class_ = 'mw-page-title-main').text
        print(player_name)
    else:
        player_name = player_soup.find('span', class_ = 'note').text
    new_dict = {}
    new_dict['Player'] = player_name
    print(player_name)
    if player_soup.find('th', text = lambda t: t != None and('MLB statistics' in t or 'Career statistics' in t)) == None:
        start = player_soup.find('th').parent
        for i in start.find_next_siblings():    
                stat_text = i.find('th',)
                stat_num =i.find('td')
                if i.find('th', text = 'Teams'):
                    break
                if stat_text != None and stat_num != None:
                    key = stat_text.text
                    value = stat_num.text
                    new_dict[key] = value
                    print(key)
                    print(value)
        player_list.append(new_dict)
    else:           
        start = player_soup.find('th', text = lambda t: t != None and('MLB statistics' in t or 'Career statistics' in t)).parent
        for tr in start.find_next_siblings('tr'):
            if tr.find('th', text = 'Teams') or tr.find('a') is None:
                break
            if type(tr.find('a')) is not None:
                key = tr.find('a').text
                value = tr.find('td').text
                new_dict[key] = value
                print(tr.find('a').text)
                print(tr.find('td').text)
        player_list.append(new_dict)

Ty Cobb
Batting average
.367
Hits
4,191
Home runs
117
Runs batted in
1,938
Stolen bases
892
Frank Schulte
Batting average
.270
Home runs
92
Runs batted in
793
Tris Speaker
Batting average
.345
Hits
3,514
Home runs
117
Runs batted in
1,529
Doubles
792
Larry Doyle
Batting average
.290
Home runs
74
Runs batted in
794
Walter Johnson
Win–loss record
417–279
Earned run average
2.17
Strikeouts
3,508
Shutouts
110
Jake Daubert
Batting average
.303
Hits
2,326
Home runs
56
Runs batted in
722
Eddie Collins
Batting average
.333
Hits
3,315
Home runs
47
Runs batted in
1,300
Stolen bases
745
Johnny Evers
Batting average
.270
Home runs
12
Runs batted in
538
Stolen bases
324
George Sisler
Batting average
.340
Hits
2,812
Home runs
102
Runs batted in
1,175
Babe Ruth
Batting average
.342
Hits
2,873
Home runs
714
Runs batted in
2,214
Win–loss record
94–46
Earned run average
2.28
Walter Johnson
Win–loss record
417–279
Earned run average
2.17
Strikeouts
3,508
Shutouts
110
Dazzy Vance
Win–loss record
197–140
E

In [31]:
# player_list = []
# for i in range(150,len(player_links)):
#     player_url = 'https://en.wikipedia.org' + str(player_links[i])
#     player_html = requests.get(player_url)
#     player_soup = BeautifulSoup(player_html.content, 'html.parser')
#     if player_soup.find('span', class_ = 'note') == None:
#         player_soup.find('span', class_ = 'mw-page-title-main').text
#         print(player_name)
#     else:
#         player_name = player_soup.find('span', class_ = 'note').text
#     new_dict = {}
#     new_dict['Player'] = player_name
#     print(player_name)
#     start = player_soup.find('th', text = lambda t: t != None and('MLB statistics' in t or 'Career statistics' in t)).parent
#     for tr in start.find_next_siblings('tr'):
#         if tr.find('th', text = 'Teams') or tr.find('a') is None:
#             break
#         if type(tr.find('a')) is not None:
#             key = tr.find('a').text
#             value = tr.find('td').text
#             new_dict[key] = value
#             print(tr.find('a').text)
#             print(tr.find('td').text)
#     player_list.append(new_dict)

Mo Vaughn
Batting average
.293
Home runs
328
Runs batted in
1,064
Barry Larkin
Batting average
.295
Hits
2,340
Home runs
198
Runs batted in
960
Stolen bases
379
Juan González
Batting average
.295
Home runs
434
Runs batted in
1,404
Ken Caminiti
Batting average
.272
Home runs
239
Runs batted in
983
Ken Griffey Jr.
Batting average
.284
Hits
2,781
Home runs
630
Runs batted in
1,836
Larry Walker
Batting average
.313
Hits
2,160
Home runs
383
Runs batted in
1,311
Juan González
Batting average
.295
Home runs
434
Runs batted in
1,404
Sammy Sosa
Batting average
.273
Hits
2,408
Home runs
609
Runs batted in
1,667
Sammy Sosa
Sammy Sosa
Batting average
.296
Hits
2,844
Home runs
311
Runs batted in
1,332
Chipper Jones
Batting average
.303
Hits
2,726
Home runs
468
Runs batted in
1,623
Jason Giambi
Batting average
.277
Hits
2,010
Home runs
440
Runs batted in
1,441
Jeff Kent
Batting average
.290
Hits
2,461
Home runs
377
Runs batted in
1,518
Ichiro Suzuki
Barry Bonds
Batting average
.298
Home runs
762
Hit

AttributeError: 'NoneType' object has no attribute 'parent'

In [74]:
print(player_list)

[{'Player': 'Ty Cobb', 'Batting average': '.367', 'Hits': '4,191', 'Home runs': '117', 'Runs batted in': '1,938', 'Stolen bases': '892'}, {'Player': 'Frank Schulte', 'Batting average': '.270', 'Home runs': '92', 'Runs batted in': '793'}, {'Player': 'Tris Speaker', 'Batting average': '.345', 'Hits': '3,514', 'Home runs': '117', 'Runs batted in': '1,529', 'Doubles': '792'}, {'Player': 'Larry Doyle', 'Batting average': '.290', 'Home runs': '74', 'Runs batted in': '794'}, {'Player': 'Walter Johnson', 'Win–loss record': '417–279', 'Earned run average': '2.17', 'Strikeouts': '3,508', 'Shutouts': '110'}, {'Player': 'Jake Daubert', 'Batting average': '.303', 'Hits': '2,326', 'Home runs': '56', 'Runs batted in': '722'}, {'Player': 'Eddie Collins', 'Batting average': '.333', 'Hits': '3,315', 'Home runs': '47', 'Runs batted in': '1,300', 'Stolen bases': '745'}, {'Player': 'Johnny Evers', 'Batting average': '.270', 'Home runs': '12', 'Runs batted in': '538', 'Stolen bases': '324'}, {'Player': 'Geo

In [87]:
##function that returns unique MVP statistics
unique_stats = {}
for i in player_list:
    for x in i.keys():
        if x not in unique_stats.keys():
            unique_stats[x] = 1
        else:
            unique_stats[x] += 1
    

In [81]:
stats_df = pd.DataFrame(player_list)
print(stats_df.columns)
print(stats_df.head())
print(stats_df)

Index(['Player', 'Batting average', 'Hits', 'Home runs', 'Runs batted in',
       'Stolen bases', 'Doubles', 'Win–loss record', 'Earned run average',
       'Strikeouts', 'Shutouts', 'On-base percentage', 'Saves', 'Runs',
       'WHIP'],
      dtype='object')
           Player Batting average   Hits Home runs Runs batted in  \
0         Ty Cobb            .367  4,191       117          1,938   
1   Frank Schulte            .270    NaN        92            793   
2    Tris Speaker            .345  3,514       117          1,529   
3     Larry Doyle            .290    NaN        74            794   
4  Walter Johnson             NaN    NaN       NaN            NaN   

  Stolen bases Doubles Win–loss record Earned run average Strikeouts Shutouts  \
0          892     NaN             NaN                NaN        NaN      NaN   
1          NaN     NaN             NaN                NaN        NaN      NaN   
2          NaN     792             NaN                NaN        NaN      NaN   
3

In [88]:
print(unique_stats)

{'Player': 204, 'Batting average': 179, 'Hits': 129, 'Home runs': 179, 'Runs batted in': 179, 'Stolen bases': 31, 'Doubles': 1, 'Win–loss record': 27, 'Earned run average': 27, 'Strikeouts': 26, 'Shutouts': 2, 'On-base percentage': 2, 'Saves': 4, 'Runs': 1, 'WHIP': 1}


In [53]:
player_url = 'https://en.wikipedia.org/wiki/Joey_Votto'
player_html = requests.get(player_url)
player_soup = BeautifulSoup(player_html.content, 'html.parser')

In [64]:
player_soup.find('span', class_ = 'mw-page-title-main').text
for i in player_soup.find_all('th'):
    pass
    # print(i.text)
start = player_soup.find('th').parent
print(start)
for i in start.find_next_siblings():    
        stat_text = i.find('th',)
        stat_num =i.find('td')
        if stat_text != None and stat_num != None:
            print(stat_text.text)
            print(stat_num.text)
# for tr in start.find_next_siblings('tr'):
#     if tr.find('th', text = 'Teams') or tr.find('a') is None:
#         break
#     if type(tr.find('a')) is not None:
#             key = tr.find('a').text
#             value = tr.find('td').text
#             new_dict[key] = value
#             print(tr.find('a').text)
#             print(tr.find('td').text)

<tr><th class="infobox-above fn" colspan="2" style="background-color: #C6011F; color: #FFFFFF;"><span class="note">Joey Votto</span><link href="mw-data:TemplateStyles:r1066479718" rel="mw-deduplicated-inline-style"/></th></tr>
Batting average
.297
Hits
2,093
Home runs
342
Runs batted in
1,106


In [93]:
csv_path = Path('csv_data\stats.csv')
csv_path.parent.mkdir(parents = True, exist_ok = True)
stats_csv = stats_df.to_csv(csv_path, index = False)

In [94]:
mvp_path = Path('csv_data\mvp.csv')
mvp_path.parent.mkdir(parents = True, exist_ok = True)
mvps_csv = new_df.to_csv(mvp_path, index = False)